<a href="https://colab.research.google.com/github/Pierre-LouisDeu/gaussian-splatting/blob/main/gsplat_colab_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Upload your dataset in Google Drive and mount in Colab. (If you want to train your own dataset)
1. Check if the selected GPU has CUDA.

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
    print("CUDA version:", torch.version.cuda)
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")


CUDA is available.
CUDA version: 12.1
GPU: Tesla T4


2. Clone Gaussian Splatting repository.

In [1]:
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q /content/gaussian-splatting/submodules/diff-gaussian-rasterization
!pip install -q /content/gaussian-splatting/submodules/simple-knn

/content
Cloning into 'gaussian-splatting'...
remote: Enumerating objects: 603, done.
remote: Total 603 (delta 0), reused 0 (delta 0), pack-reused 603 (from 1)
Receiving objects: 100% (603/603), 2.09 MiB | 32.95 MiB/s, done.
Resolving deltas: 100% (347/347), done.
Submodule 'SIBR_viewers' (https://gitlab.inria.fr/sibr/sibr_core) registered for path 'SIBR_viewers'
Submodule 'submodules/diff-gaussian-rasterization' (https://github.com/graphdeco-inria/diff-gaussian-rasterization) registered for path 'submodules/diff-gaussian-rasterization'
Submodule 'submodules/simple-knn' (https://gitlab.inria.fr/bkerbl/simple-knn.git) registered for path 'submodules/simple-knn'
Cloning into '/content/gaussian-splatting/SIBR_viewers'...
remote: Enumerating objects: 3270, done.        
remote: Counting objects: 100% (299/299), done.        
remote: Compressing objects: 100% (164/164), done.        
remote: Total 3270 (delta 157), reused 259 (delta 135), pack-reused 2971 (from 1)        
Receiving objects:

3. Fetch sample data. Optional if you have your own dataset and mounted it via Drive. Change the name of /images folder to /input. Ex: .../db/playroom/images => .../db/playroom/input

In [2]:
!wget https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
!unzip tandt_db.zip

--2024-12-11 22:09:26--  https://huggingface.co/camenduru/gaussian-splatting/resolve/main/tandt_db.zip
Resolving huggingface.co (huggingface.co)... 13.35.210.66, 13.35.210.114, 13.35.210.61, ...
Connecting to huggingface.co (huggingface.co)|13.35.210.66|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/69/11/6911555bcb5edae447d514169831329398b45a3ed9ae3b00817878bc501559a1/816e62f22a161abbfe841d2a6b10cdf036e297c9fa289b3bfeee9c6ec526d7e1?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tandt_db.zip%3B+filename%3D%22tandt_db.zip%22%3B&response-content-type=application%2Fzip&Expires=1734214166&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNDIxNDE2Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82OS8xMS82OTExNTU1YmNiNWVkYWU0NDdkNTE0MTY5ODMxMzI5Mzk4YjQ1YTNlZDlhZTNiMDA4MTc4NzhiYzUwMTU1OWExLzgxNmU2MmYyMmExNjFhYmJmZTg0MWQyYTZiMTBjZGYwMzZlMjk3YzlmYTI4OWIzYmZlZWU5YzZlYzUyNm

4. Install dependencies for COLMAP:

In [3]:
!sudo apt-get install -y \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-regex-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev \
    libsuitesparse-dev \
    libgflags-dev \
    libpng-dev \
    libjpeg-dev \
    libtiff-dev \
    libxxf86vm1 \
    libxxf86vm-dev \
    libxi-dev \
    libxrandr-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libboost-filesystem-dev is already the newest version (1.74.0.3ubuntu7).
libboost-filesystem-dev set to manually installed.
libboost-program-options-dev is already the newest version (1.74.0.3ubuntu7).
libboost-program-options-dev set to manually installed.
libboost-regex-dev is already the newest version (1.74.0.3ubuntu7).
libboost-regex-dev set to manually installed.
libboost-system-dev is already the newest version (1.74.0.3ubuntu7).
libboost-system-dev set to manually installed.
libjpeg-dev is already the newest version (8c-2ubuntu10).
libjpeg-dev set to manually installed.
libpng-dev is already the newest version (1.6.37-3build5).
libxxf86vm1 is already the newest version (1:1.1.4-1build3).
libxxf86vm1 set to manually installed.
libboost-graph-dev is already the newest version (1.74.0.3ubuntu7).
libboost-graph-dev set to manu

5. Build COLMAP. Takes ~30 minutes.

In [4]:
%cd /content
!git clone https://github.com/colmap/colmap.git
%cd colmap
!mkdir build
%cd build
!cmake ..
!make -j$(nproc)
!sudo make install

/content
Cloning into 'colmap'...
remote: Enumerating objects: 23873, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 23873 (delta 37), reused 68 (delta 33), pack-reused 23794 (from 1)
Receiving objects: 100% (23873/23873), 69.57 MiB | 14.47 MiB/s, done.
Resolving deltas: 100% (18477/18477), done.
/content/colmap
/content/colmap/build
-- Enabling LSD support
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMake Error at cmake/FindDependencies.cmake:16 (find_package):
  By

5.5. Download exemple dataset  

In [ ]:
# !python https://raw.githubusercontent.com/nerfstudio-project/gsplat/refs/heads/main/examples/datasets/download_dataset.py

6. Convert the dataset using COLMAP. <br />
Change the path to your own dataset or use the sample if you just want to try things out.

In [5]:
%cd /content/gaussian-splatting
!python convert.py -s /content/gaussian-splatting/db/playroom/

/content/gaussian-splatting
sh: 1: colmap: not found
ERROR:root:Feature extraction failed with code 32512. Exiting.


7. Train the converted dataset.

In [ ]:
# Replace {path_to_dataset}
# --save_iterations: arg for saved checkpoints. Set different number if you want.
!python train.py -s /content/gaussian-splatting/db/playroom --iterations 30000 --save_iterations 10000 20000 30000   #/content/gaussian-splatting/db/playroom

2024-12-11 22:11:34.174684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 22:11:34.206509: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 22:11:34.216127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 22:11:34.238104: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-11 22:11:35.722421: W tensorflow/comp

8. Download the .ply file in .../output/{sha}/point_cloud/{iteration_num}/point_cloud.py